# Task 3: MiniFlow 

<img src='utils/addition-graph.png' width="200" height="100">

In [ ]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        self.value = None
        for n in inbound_nodes:
            n.outbound_nodes.append(self)      # What is "self here?
            
    def forward(self):
        raise NotImplemented

In [ ]:
# Input node does not have inbound nodes (thus an empty array). 

class Input(Node):
    def __init__(self):
        Node.__init__(self)                   # Why do we introduce "self", and leave "inbound_nodes" behind? 
                                              # In this case, inboud nodes is by default initialized as an empty array?
    def forward(self, value=None):
        if value is not None:
            self.value = value

In [ ]:
# Create a node that would perform the operation of addition on inbound nodes. 

class Add(Node):
    def __init__(self, x, y):
        Node.__init__(self, [x, y])
        
    def forward(self):
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value

In [ ]:
# Perform a topological sort of nodes using Kahn's algorithm

def topological_sort(feed_dict):
    """
    Performs a topological sort using Kahn's algorithm.
    
    Argument:
    'feed_dict': the key is an input node, the value is value of the node
    
    Returns a list.
    """
    
    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)                                 # Remove indicated index
        if n not in G:
            G[n] = {'in': set(), 'out': set()}  
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [ ]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [ ]:
# Execute 
x, y = Input(), Input()
f = Add(x, y)
feed_dict = {x: 10, y: 5}
sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)
print("{} + {} = {} (according to MiniFlow)".format(feed_dict[x], feed_dict[y], output))